In [ ]:
#!/usr/bin/env python3
"""
Greedy Ride Matcher - Python
Matches pending passengers (public_ride_requests) -> riders (riders)
Writes proposals to rider DB collection driver_proposals.

Notes:
- Requires firebase_admin with access to service account JSON files.
- Uses a real-time Firestore listener on passenger DB (pending requests).
- Added automatic passenger status progression based on driver/proposal actions:
    pending -> accepted -> arrived_at_pickup -> picked_up -> on_way -> completed
"""
import os
import time
import traceback
from math import radians, sin, cos, sqrt, atan2

import firebase_admin
from firebase_admin import credentials, firestore
from firebase_admin.firestore import GeoPoint

# ----------------- CONFIG -----------------
PASSENGER_DB_CREDENTIALS = os.environ.get(
    "PASSENGER_DB_CREDENTIALS",
    "passenger-ride-app-firebase-adminsdk-fbsvc-1061e4a556.json",
)
RIDER_DB_CREDENTIALS = os.environ.get(
    "RIDER_DB_CREDENTIALS",
    "rider-ba88e-firebase-adminsdk-fbsvc-57d40ed3f7.json",
)

PASSENGER_REQUESTS_COL = "public_ride_requests"
RIDERS_COL = "riders"
DRIVER_PROPOSALS_COL = "driver_proposals"

MAX_MATCH_DISTANCE_KM = float(os.environ.get("MAX_MATCH_DISTANCE_KM", 5.0))
MAX_DESTINATION_DEVIATION_KM = float(os.environ.get("MAX_DESTINATION_DEVIATION_KM", 5.0))

ELIGIBLE_DRIVER_STATUSES = (
    "on_route_to_original_destination",
    "available",
    "idle",
    "on_route_to_pickup",
    "en_route",
)

# How close driver must be to count as "arrived" (km)
ARRIVED_DISTANCE_THRESHOLD_KM = float(os.environ.get("ARRIVED_DISTANCE_THRESHOLD_KM", 0.05))  # 50 meters

# ----------------- Utilities -----------------

def log(msg: str):
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] {msg}")

def init_firestore_app(cred_path: str, name: str):
    """Initialize Firestore client for a Firebase app."""
    if not os.path.exists(cred_path):
        log(f"ERROR: credential file not found: {cred_path}")
        return None
    try:
        cred = credentials.Certificate(cred_path)
        try:
            app = firebase_admin.get_app(name)
            log(f"Re-using Firebase app '{name}'.")
        except ValueError:
            app = firebase_admin.initialize_app(cred, name=name)
            log(f"Initialized Firebase app '{name}'.")
        return firestore.client(app=app)
    except Exception as e:
        log(f"Error initializing Firebase app '{name}': {e}")
        traceback.print_exc()
        return None

def haversine_km(lat1, lon1, lat2, lon2):
    """Haversine distance between two points in kilometers."""
    lat1_rad, lon1_rad, lat2_rad, lon2_rad = map(radians, map(float, [lat1, lon1, lat2, lon2]))
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = sin(dlat / 2) ** 2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2) ** 2
    return 2 * 6371.0 * atan2(sqrt(a), sqrt(1 - a))

def to_geopoint(loc):
    """Normalize input to a Firestore GeoPoint."""
    if loc is None:
        return None
    if isinstance(loc, GeoPoint):
        return loc
    if isinstance(loc, dict):
        lat = next((loc.get(k) for k in ("latitude", "lat", "Latitude") if loc.get(k) is not None), None)
        lon = next((loc.get(k) for k in ("longitude", "lng", "lon", "Longitude") if loc.get(k) is not None), None)
        for cand in ("coords", "location", "geo", "position"):
            if cand in loc and isinstance(loc[cand], dict):
                nested = loc[cand]
                lat = nested.get("latitude") or nested.get("lat") or lat
                lon = nested.get("longitude") or nested.get("lng") or nested.get("lon") or lon
                if lat is not None and lon is not None:
                    break
        if lat is not None and lon is not None:
            try:
                return GeoPoint(float(lat), float(lon))
            except Exception:
                return None
    if hasattr(loc, "latitude") and hasattr(loc, "longitude"):
        try:
            return GeoPoint(float(loc.latitude), float(loc.longitude))
        except Exception:
            return None
    return None

def calculate_incremental_detour_km(driver_start, driver_end, pickup, dropoff):
    """Incremental detour in km if driver picks up this passenger."""
    base = haversine_km(driver_start.latitude, driver_start.longitude, driver_end.latitude, driver_end.longitude)
    new_total = (
        haversine_km(driver_start.latitude, driver_start.longitude, pickup.latitude, pickup.longitude)
        + haversine_km(pickup.latitude, pickup.longitude, dropoff.latitude, dropoff.longitude)
        + haversine_km(dropoff.latitude, dropoff.longitude, driver_end.latitude, driver_end.longitude)
    )
    return new_total - base, base, new_total

# ----------------- Matching Logic -----------------

def create_proposal_payload(passenger_doc, passenger_data, driver_doc_id, driver_data, final_pickup):
    """Prepare proposal payload with all necessary info for rider."""
    passenger_uid = passenger_data.get("passengerId") or passenger_data.get("passengerUid") or passenger_data.get("riderUid")
    pickup_loc = to_geopoint(passenger_data.get("pickupLocation") or passenger_data.get("pickup_location") or passenger_data.get("pickup"))
    dest_loc = to_geopoint(passenger_data.get("destinationLocation") or passenger_data.get("destination") or passenger_data.get("destination_location"))

    driver_loc = to_geopoint(driver_data.get("currentRouteStart") or driver_data.get("nextTargetLocation") or driver_data.get("current_location") or driver_data.get("currentLocation"))

    payload = {
        "request_id": passenger_doc.id,
        "status": "pending_acceptance",
        "createdAt": firestore.SERVER_TIMESTAMP,
        # --- Passenger Info ---
        "passengerUid": passenger_uid,
        "passengerName": passenger_data.get("passengerName") or passenger_data.get("name") or "Unknown Passenger",
        "passengerPhone": passenger_data.get("passengerPhone") or passenger_data.get("phone") or "Not Provided",
        "pickupLocation": final_pickup,
        "destinationLocation": dest_loc,
        "pickup_address": passenger_data.get("pickupAddress") or passenger_data.get("pickup_address"),
        "destination_address": passenger_data.get("destinationAddress") or passenger_data.get("destination_address"),
        "fareAmount": passenger_data.get("fareAmount") or 0.0,
        "paymentMethod": passenger_data.get("paymentMethod") or "Cash",
        "rideType": passenger_data.get("rideType") or "Standard",
        "passengerRating": passenger_data.get("passengerRating") or 5.0,
        "estimatedDistance": passenger_data.get("estimatedDistance") or "N/A",
        "estimatedDuration": passenger_data.get("estimatedDuration") or "N/A",
        "specialRequests": passenger_data.get("specialRequests") or "None",
        "vehiclePreference": passenger_data.get("vehiclePreference") or "Any",
        "luggageCount": passenger_data.get("luggageCount") or 0,
        "passengerCount": passenger_data.get("passengerCount") or 1,
        "otp": passenger_data.get("otp") or "0000",
        "otpVerified": passenger_data.get("otpVerified") or False,
        "sosActive": passenger_data.get("sosActive") or False,
        "sosReason": passenger_data.get("sosReason"),
        "sosTimestamp": passenger_data.get("sosTimestamp"),
        # --- Driver Info ---
        "riderUid": driver_data.get("uid") or driver_doc_id,
        "driverId": driver_data.get("uid") or driver_doc_id,
        "driverName": driver_data.get("name") or driver_data.get("driverName") or "Unknown Driver",
        "driverPhone": driver_data.get("phone") or "Not Provided",
        "driverVehicle": driver_data.get("vehicleType") or "Unknown Vehicle",
        "riderLocation": driver_loc,
        "lastLocationUpdate": firestore.SERVER_TIMESTAMP,
        # --- Route Info ---
        "routeToPickupEncoded": passenger_data.get("routeToPickupEncoded"),
        "routeToDestinationEncoded": passenger_data.get("routeToDestinationEncoded"),
        # --- Timestamps ---
        "acceptedTimestamp": None,
        "arrivalTimestamp": None,
        "pickupTimestamp": None,
        "completionTimestamp": None,
        "cancellationTimestamp": None,
        "distanceToPickup": None,
    }

    # compute distanceToPickup safely
    try:
        if pickup_loc and driver_loc:
            payload["distanceToPickup"] = haversine_km(
                pickup_loc.latitude, pickup_loc.longitude,
                driver_loc.latitude, driver_loc.longitude
            )
    except Exception:
        payload["distanceToPickup"] = None

    return payload

def try_reserve_driver_and_create_proposal(rider_db, passenger_db, passenger_doc, driver_doc_id, driver_data, proposal_payload):
    """Reserve driver and write proposal atomically, then update passenger doc."""
    rider_ref = rider_db.collection(RIDERS_COL).document(driver_doc_id)
    proposal_ref = rider_db.collection(DRIVER_PROPOSALS_COL).document()

    @firestore.transactional
    def txn_reserve(transaction):
        snapshot = rider_ref.get(transaction=transaction)
        if not snapshot.exists:
            raise RuntimeError("Driver doc disappeared during reservation.")
        if snapshot.get("status") not in ELIGIBLE_DRIVER_STATUSES:
            raise RuntimeError(f"Driver {driver_doc_id} status not eligible.")
        transaction.update(rider_ref, {"status": "reserved_for_proposal", "reserved_for_request": passenger_doc.id})
        transaction.set(proposal_ref, proposal_payload)

    try:
        transaction = rider_db.transaction()
        txn_reserve(transaction)
    except Exception as e:
        log(f"Failed to reserve driver {driver_doc_id}: {e}")
        traceback.print_exc()
        return False, None

    try:
        # Update passenger with full driver info for Flutter to read directly
        passenger_db.collection(PASSENGER_REQUESTS_COL).document(passenger_doc.id).update({
            "status": "proposed",
            "riderUid": driver_data.get("uid") or driver_doc_id,
            "riderName": driver_data.get("name") or "Unknown Driver",
            "riderPhone": driver_data.get("phone") or "Not Provided",
            "riderLocation": driver_data.get("current_location") or driver_data.get("currentLocation"),
            "matchedDriverName": driver_data.get("name") or "Unknown Driver",
            "matchedDriverPhone": driver_data.get("phone") or "Not Provided",
            "matchedDriverVehicle": driver_data.get("vehicleType") or "Unknown Vehicle",
            "proposed_at": firestore.SERVER_TIMESTAMP,
        })
        log(f"Passenger {passenger_doc.id} updated to 'proposed' with driver {driver_doc_id}.")
        return True, proposal_ref.id
    except Exception as e:
        log(f"Failed to update passenger {passenger_doc.id}: {e}")
        traceback.print_exc()
        try:
            rider_db.collection(RIDERS_COL).document(driver_doc_id).update(
                {"status": driver_data.get("status", "available"), "reserved_for_request": firestore.DELETE_FIELD}
            )
        except Exception:
            log(f"Warning: failed to revert reservation for driver {driver_doc_id}.")
        return False, None

def match_one_request(passenger_db, rider_db, passenger_doc):
    """Match a single passenger request to the best available driver."""
    try:
        passenger_data = passenger_doc.to_dict() or {}
        if not passenger_data or passenger_data.get("status", "").lower() != "pending":
            return

        pickup = to_geopoint(passenger_data.get("pickupLocation") or passenger_data.get("pickup_location") or passenger_data.get("pickup"))
        dest = to_geopoint(passenger_data.get("destinationLocation") or passenger_data.get("destination") or passenger_data.get("destination_location"))
        vehicle_pref = passenger_data.get("vehiclePreference") or "Any"

        if not (pickup and dest):
            log(f"Passenger {passenger_doc.id}: pickup/destination invalid - skipping.")
            return

        log(f"Matching passenger {passenger_doc.id} at ({pickup.latitude:.6f},{pickup.longitude:.6f})")

        riders_q = rider_db.collection(RIDERS_COL).where("status", "in", list(ELIGIBLE_DRIVER_STATUSES))
        riders = list(riders_q.stream())
        if not riders:
            log("No eligible drivers available.")
            return

        best = None
        best_cost = float("inf")
        best_pickup_dist = float("inf")

        for rdoc in riders:
            try:
                rdata = rdoc.to_dict() or {}
                driver_start = to_geopoint(rdata.get("currentRouteStart") or rdata.get("nextTargetLocation") or rdata.get("current_location") or rdata.get("currentLocation"))
                driver_end = to_geopoint(rdata.get("currentRouteEnd") or rdata.get("destination") or rdata.get("currentDestination"))

                if not (driver_start and driver_end):
                    continue

                driver_vehicle = (rdata.get("vehicleType") or "").lower()
                if vehicle_pref != "Any" and vehicle_pref.lower() not in driver_vehicle:
                    continue

                pickup_dist = haversine_km(driver_start.latitude, driver_start.longitude, pickup.latitude, pickup.longitude)
                if pickup_dist > MAX_MATCH_DISTANCE_KM:
                    continue

                dest_dev = haversine_km(driver_end.latitude, driver_end.longitude, dest.latitude, dest.longitude)
                if dest_dev > MAX_DESTINATION_DEVIATION_KM:
                    continue

                inc_detour, _, _ = calculate_incremental_detour_km(driver_start, driver_end, pickup, dest)
                if inc_detour < best_cost or (abs(inc_detour - best_cost) < 1e-6 and pickup_dist < best_pickup_dist):
                    best_cost = inc_detour
                    best = (rdoc.id, rdata, driver_start, driver_end, pickup_dist, inc_detour)
                    best_pickup_dist = pickup_dist
            except Exception:
                log("Warning: error evaluating driver; skipping.")
                traceback.print_exc()

        if not best:
            log(f"No suitable driver for passenger {passenger_doc.id}.")
            return

        driver_doc_id, driver_data, driver_start, driver_end, pickup_dist, inc_detour = best
        log(f"Selected driver {driver_doc_id}: pickup_dist={pickup_dist:.3f} km, incremental_detour={inc_detour:.3f} km")

        final_pickup = GeoPoint((driver_start.latitude + pickup.latitude) / 2, (driver_start.longitude + pickup.longitude) / 2) if pickup_dist < 2.0 else pickup

        proposal_payload = create_proposal_payload(passenger_doc, passenger_data, driver_doc_id, driver_data, final_pickup)
        success, proposal_id = try_reserve_driver_and_create_proposal(rider_db, passenger_db, passenger_doc, driver_doc_id, driver_data, proposal_payload)

        if success:
            log(f"Proposal created (id={proposal_id}) for passenger {passenger_doc.id} -> driver {driver_doc_id}")
        else:
            log(f"Failed to finalize proposal for passenger {passenger_doc.id}")

    except Exception as e:
        log(f"Error matching passenger {passenger_doc.id}: {e}")
        traceback.print_exc()

# ----------------- Status mapping helpers -----------------

def _safe_get(data, *keys, default=None):
    for k in keys:
        if data is None:
            break
        if isinstance(data, dict) and k in data:
            return data.get(k)
    return default

def map_proposal_status_to_passenger(proposal_status):
    """Map a driver_proposal.status to a passenger request status."""
    ps = (proposal_status or "").lower()
    if ps in ("accepted", "driver_accepted"):
        return "accepted"
    if ps in ("driver_arrived", "arrived", "arrived_at_pickup"):
        return "arrived_at_pickup"
    if ps in ("otp_verified",) :
        # OTP alone might not mean picked; wait for face verification or picked_up explicit.
        return "otp_verified"
    if ps in ("face_verified",):
        return "face_verified"
    if ps in ("picked_up", "pickedup"):
        return "picked_up"
    if ps in ("on_way", "on_the_way", "en_route"):
        return "on_way"
    if ps in ("completed", "finished"):
        return "completed"
    if ps in ("rejected", "cancelled", "declined"):
        return "rejected"
    return None

# ----------------- Driver Proposal Listener (progress updates) -----------------

def listen_for_driver_proposal_progress(rider_db, passenger_db):
    """Listen for updates on driver proposals and propagate to passenger request statuses."""
    # watch for proposals changing to statuses we care about
    interesting_statuses = [
        "accepted", "driver_arrived", "otp_verified", "face_verified",
        "picked_up", "on_way", "completed", "rejected", "arrived", "pickedup", "driver_accepted", "finished"
    ]

    def on_proposals_snapshot(col_snapshot, changes, read_time):
        for change in changes:
            try:
                doc = change.document
                data = doc.to_dict() or {}
                # only handle modifications (status transitions)
                if change.type.name not in ("ADDED", "MODIFIED"):
                    continue

                proposal_status = (data.get("status") or "").lower()
                mapped = map_proposal_status_to_passenger(proposal_status)
                request_id = data.get("request_id") or data.get("requestId") or data.get("request")
                rider_uid = data.get("riderUid") or data.get("driverId") or data.get("riderId") or doc.get("riderUid")

                if not request_id:
                    # nothing to update
                    continue

                if mapped is None:
                    # status not recognized, skip
                    continue

                # Special handling for some intermediate states:
                if mapped == "otp_verified":
                    # mark otpVerified on passenger; do not change status to picked_up yet
                    try:
                        passenger_db.collection(PASSENGER_REQUESTS_COL).document(request_id).update({
                            "otpVerified": True,
                            "otp_verified_at": firestore.SERVER_TIMESTAMP,
                        })
                        log(f"Passenger {request_id}: OTP verified (proposal {doc.id}).")
                    except Exception as e:
                        log(f"Failed to set otpVerified for {request_id}: {e}")
                        traceback.print_exc()
                    continue

                if mapped == "face_verified":
                    # mark faceVerified; if both otp & face verified, mark picked_up only if driver indicates pickup
                    try:
                        passenger_db.collection(PASSENGER_REQUESTS_COL).document(request_id).update({
                            "faceVerified": True,
                            "face_verified_at": firestore.SERVER_TIMESTAMP,
                        })
                        log(f"Passenger {request_id}: Face verified (proposal {doc.id}).")
                    except Exception as e:
                        log(f"Failed to set faceVerified for {request_id}: {e}")
                        traceback.print_exc()
                    continue

                if mapped == "rejected":
                    # Proposal rejected by driver - revert passenger to 'pending' or 'no_driver' (app logic)
                    try:
                        passenger_db.collection(PASSENGER_REQUESTS_COL).document(request_id).update({
                            "status": "pending",
                            "proposed_driver": firestore.DELETE_FIELD,
                            "riderUid": firestore.DELETE_FIELD,
                            "matchedDriverName": firestore.DELETE_FIELD,
                            "matchedDriverPhone": firestore.DELETE_FIELD,
                            "matchedDriverVehicle": firestore.DELETE_FIELD,
                            "proposed_at": firestore.DELETE_FIELD,
                        })
                        # free rider reservation if rider uid present
                        if rider_uid:
                            try:
                                rider_db.collection(RIDERS_COL).document(rider_uid).update({
                                    "status": "available",
                                    "reserved_for_request": firestore.DELETE_FIELD
                                })
                            except Exception:
                                log(f"Warning: failed to clear reservation for rider {rider_uid}")
                        log(f"Passenger {request_id}: proposal rejected; reverted to pending.")
                    except Exception as e:
                        log(f"Failed to revert passenger {request_id} after proposal rejection: {e}")
                        traceback.print_exc()
                    continue

                # For arrival, picked_up, on_way, completed -> set passenger status accordingly and add timestamps
                update_payload = {}
                ts_field = None

                if mapped == "accepted":
                    update_payload["status"] = "accepted"
                    update_payload["accepted_at"] = firestore.SERVER_TIMESTAMP
                    ts_field = "accepted_at"
                elif mapped == "arrived_at_pickup":
                    update_payload["status"] = "arrived_at_pickup"
                    update_payload["arrived_at"] = firestore.SERVER_TIMESTAMP
                    ts_field = "arrived_at"
                elif mapped == "picked_up":
                    update_payload["status"] = "picked_up"
                    update_payload["pickupTimestamp"] = firestore.SERVER_TIMESTAMP
                    update_payload["picked_up_at"] = firestore.SERVER_TIMESTAMP
                    ts_field = "picked_up_at"
                elif mapped == "on_way":
                    update_payload["status"] = "on_way"
                    update_payload["on_way_at"] = firestore.SERVER_TIMESTAMP
                    ts_field = "on_way_at"
                elif mapped == "completed":
                    update_payload["status"] = "completed"
                    update_payload["completed_at"] = firestore.SERVER_TIMESTAMP
                    ts_field = "completed_at"

                # Add driver summary fields so passenger UI can display live info
                driver_name = data.get("driverName") or data.get("driver_name") or data.get("driver") or data.get("driverFullName")
                driver_phone = data.get("driverPhone") or data.get("driver_phone") or data.get("driver_contact")
                rider_location = data.get("riderLocation") or data.get("driverLocation") or data.get("rider_location")

                if driver_name:
                    update_payload["matchedDriverName"] = driver_name
                if driver_phone:
                    update_payload["matchedDriverPhone"] = driver_phone
                if rider_uid:
                    update_payload["riderUid"] = rider_uid
                    update_payload["riderId"] = rider_uid

                if rider_location:
                    update_payload["riderLocation"] = rider_location
                    update_payload["lastLocationUpdate"] = firestore.SERVER_TIMESTAMP

                # apply update
                try:
                    passenger_db.collection(PASSENGER_REQUESTS_COL).document(request_id).update(update_payload)
                    log(f"Passenger {request_id} updated to '{update_payload.get('status')}' (proposal {doc.id}).")
                except Exception as e:
                    log(f"Failed to update passenger {request_id} for proposal {doc.id}: {e}")
                    traceback.print_exc()

                # Also update driver/rider doc statuses to keep them in sync
                try:
                    if rider_uid and mapped in ("accepted", "arrived_at_pickup", "picked_up", "on_way", "completed"):
                        # derive driver status mapping
                        driver_status_map = {
                            "accepted": "on_route_to_pickup",
                            "arrived_at_pickup": "on_site_pickup",
                            "picked_up": "en_route",
                            "on_way": "en_route",
                            "completed": "idle",
                        }
                        new_driver_status = driver_status_map.get(mapped)
                        if new_driver_status:
                            rider_db.collection(RIDERS_COL).document(rider_uid).update({
                                "status": new_driver_status,
                                "current_ride_request": request_id if mapped != "completed" else firestore.DELETE_FIELD,
                            })
                except Exception:
                    log(f"Warning: failed to sync rider doc for rider {rider_uid}")

            except Exception as e:
                log(f"Error processing proposal snapshot change: {e}")
                traceback.print_exc()

    try:
        query = rider_db.collection(DRIVER_PROPOSALS_COL).where("status", "in", interesting_statuses)
        query.on_snapshot(on_proposals_snapshot)
        log("Driver proposal progress listener attached.")
    except Exception as e:
        log(f"Failed to attach driver proposal progress listener: {e}")
        traceback.print_exc()

# ----------------- Rider Location Updates -----------------

def listen_for_rider_location_updates(rider_db, passenger_db):
    """Listen for rider location updates and update passenger request in real-time.
    Also optionally detect 'arrived' if driver gets close to pickupLocation (best-effort).
    """
    def on_rider_location_snapshot(col_snapshot, changes, read_time):
        for change in changes:
            try:
                if change.type.name in ("MODIFIED", "ADDED"):
                    doc = change.document
                    data = doc.to_dict() or {}

                    rider_uid = doc.id
                    current_location = data.get("currentLocation") or data.get("current_location") or data.get("riderLocation")
                    current_ride_request = data.get("current_ride_request") or data.get("currentRideRequest") or data.get("current_ride")

                    if not current_location:
                        continue

                    # update passenger request last known location for UI
                    if current_ride_request:
                        try:
                            passenger_db.collection(PASSENGER_REQUESTS_COL).document(current_ride_request).update({
                                "riderLocation": current_location,
                                "lastLocationUpdate": firestore.SERVER_TIMESTAMP,
                            })
                            log(f"Updated rider location for request {current_ride_request}")
                        except Exception as e:
                            log(f"Failed to update passenger ride location for {current_ride_request}: {e}")
                            traceback.print_exc()

                    # BEST-EFFORT: detect arrival if rider has no explicit proposal 'arrived' but driver comes very close to pickup
                    try:
                        # only attempt when driver indicates en route to pickup or similar
                        driver_status = data.get("status", "").lower()
                        if current_ride_request and driver_status in ("on_route_to_pickup", "on_route_to_original_destination", "on_route_to_pickup", "reserved_for_proposal"):
                            # fetch passenger pickup point
                            pdoc = passenger_db.collection(PASSENGER_REQUESTS_COL).document(current_ride_request).get()
                            if pdoc.exists:
                                        p = pdoc.to_dict() or {}
                                        pickup = to_geopoint(p.get("pickupLocation") or p.get("pickup_location") or p.get("pickup"))
                                        if pickup:
                                            # compute distance
                                            try:
                                                drv_lat = current_location.get("latitude") or current_location.get("lat") or current_location.get("Latitude")
                                                drv_lon = current_location.get("longitude") or current_location.get("lng") or current_location.get("lon") or current_location.get("Longitude")
                                                if drv_lat is not None and drv_lon is not None:
                                                    dist = haversine_km(float(drv_lat), float(drv_lon), pickup.latitude, pickup.longitude)
                                                    if dist <= ARRIVED_DISTANCE_THRESHOLD_KM:
                                                        # set passenger to arrived_at_pickup if not already
                                                        passenger_db.collection(PASSENGER_REQUESTS_COL).document(current_ride_request).update({
                                                            "status": "arrived_at_pickup",
                                                            "arrived_at": firestore.SERVER_TIMESTAMP,
                                                        })
                                                        log(f"Auto-marked passenger {current_ride_request} as 'arrived_at_pickup' (driver {rider_uid} within {dist:.3f} km).")
                                            except Exception:
                                                # if anything fails, ignore best-effort arrival detection
                                                pass
                    except Exception:
                        pass

            except Exception as e:
                log(f"Error processing rider location update: {e}")
                traceback.print_exc()

    try:
        query = rider_db.collection(RIDERS_COL)
        query.on_snapshot(on_rider_location_snapshot)
        log("Rider location update listener attached.")
    except Exception as e:
        log(f"Failed to attach rider location listener: {e}")
        traceback.print_exc()

# ----------------- Firestore Listener for Pending Requests -----------------

def on_pending_requests_snapshot(col_snapshot, changes, read_time):
    """Firestore listener callback for new or modified pending requests."""
    for change in changes:
        try:
            if change.type.name in ("ADDED", "MODIFIED"):
                doc = change.document
                data = doc.to_dict() or {}
                if data.get("status", "").lower() == "pending_acceptance":
                    match_one_request(db_passenger, db_rider, doc)
        except Exception:
            log("Error in snapshot handler.")
            traceback.print_exc()

# ----------------- Main -----------------

if __name__ == "__main__":
    db_passenger = init_firestore_app(PASSENGER_DB_CREDENTIALS, "passenger_app")
    db_rider = init_firestore_app(RIDER_DB_CREDENTIALS, "rider_app")

    if not db_passenger or not db_rider:
        log("FATAL: Firestore clients failed to initialize. Exiting.")
        raise SystemExit(1)

    log(f"Matcher: Listening to passenger requests ({PASSENGER_REQUESTS_COL})")

    try:
        # Start all listeners
        query = db_passenger.collection(PASSENGER_REQUESTS_COL).where("status", "==", "pending")
        query.on_snapshot(on_pending_requests_snapshot)

        # Listen for driver proposal progress (accept / arrived / otp / face / picked / on_way / completed / rejected)
        listen_for_driver_proposal_progress(db_rider, db_passenger)

        # Listen for rider location updates (and best-effort arrive detection)
        listen_for_rider_location_updates(db_rider, db_passenger)

        log("All listeners attached. Running... (Ctrl+C to stop)")

        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        log("Shutting down (KeyboardInterrupt).")
    except Exception as e:
        log(f"FATAL error in main loop: {e}")
        traceback.print_exc()


[2025-10-16 12:23:31] Initialized Firebase app 'passenger_app'.
[2025-10-16 12:23:31] Initialized Firebase app 'rider_app'.
[2025-10-16 12:23:31] Matcher: Listening to passenger requests (public_ride_requests)


C:\Users\DELL\anaconda3\Lib\site-packages\google\cloud\firestore_v1\base_collection.py:304: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


[2025-10-16 12:23:31] Driver proposal progress listener attached.
[2025-10-16 12:23:31] Rider location update listener attached.
[2025-10-16 12:23:31] All listeners attached. Running... (Ctrl+C to stop)
[2025-10-16 12:23:32] Failed to revert passenger 9N2RWAC4nc4zbrJaimHb after proposal rejection: 404 No document to update: projects/passenger-ride-app/databases/(default)/documents/public_ride_requests/9N2RWAC4nc4zbrJaimHb
[2025-10-16 12:23:32] Passenger PsGJp8STqsQRDn5iIKr8 updated to 'picked_up' (proposal AkYr6EtPJLJTO8KzXgcD).


Traceback (most recent call last):
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_22212\3500454999.py", line 410, in on_proposals_snapshot
    passenger_db.collection(PASSENGER_REQUESTS_COL).document(request_id).update({
  File "C:\Users\DELL\anaconda3\Lib\site-packages\google\cloud\firestore_v1\document.py", line 325, in update
    write_results = batch.commit(**kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\DELL\anaconda3\Lib\site-packages\google\cloud\firestore_v1\batch.py", line 61, in commit
    commit_response = self._client._firestore_api.commit(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\DELL\anaconda3\Lib\site-packages\google\cloud\firestore_v1\services\firestore\client.py", line 1431, in commit
    response = rpc(
               ^^^^
  File "C:\Users\DELL\anaconda3\Lib\site-packages\google\api_core\gapic_v1\method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^

[2025-10-16 12:23:34] Failed to update passenger DGLZ6XIQ5t0zbmTfZKes for proposal zSOZXNwgD6YiTwohHi28: 404 No document to update: projects/passenger-ride-app/databases/(default)/documents/public_ride_requests/DGLZ6XIQ5t0zbmTfZKes
[2025-10-16 12:23:34] Updated rider location for request PsGJp8STqsQRDn5iIKr8


Traceback (most recent call last):
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_22212\3500454999.py", line 479, in on_proposals_snapshot
    passenger_db.collection(PASSENGER_REQUESTS_COL).document(request_id).update(update_payload)
  File "C:\Users\DELL\anaconda3\Lib\site-packages\google\cloud\firestore_v1\document.py", line 325, in update
    write_results = batch.commit(**kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\DELL\anaconda3\Lib\site-packages\google\cloud\firestore_v1\batch.py", line 61, in commit
    commit_response = self._client._firestore_api.commit(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\DELL\anaconda3\Lib\site-packages\google\cloud\firestore_v1\services\firestore\client.py", line 1431, in commit
    response = rpc(
               ^^^^
  File "C:\Users\DELL\anaconda3\Lib\site-packages\google\api_core\gapic_v1\method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore, initialize_app, get_app
from firebase_admin.firestore import GeoPoint
from math import radians, sin, cos, sqrt, atan2
import time
import os
import sys

# --- Configuration (Use Environment Variables in Production) ---
PASSENGER_DB_CREDENTIALS = 'passenger-ride-app-firebase-adminsdk-fbsvc-1061e4a556.json'
DRIVER_DB_CREDENTIALS = 'rider-ba88e-firebase-adminsdk-fbsvc-57d40ed3f7.json'

MAX_MATCH_DISTANCE_KM = 5.0
MAX_DESTINATION_DEVIATION_KM = 5.0
DYNAMIC_PICKUP_THRESHOLD_KM = 2.0

# Database Collection and Status Names
RIDER_COLLECTION_NAME = 'riders'
RIDER_MATCH_STATUS = 'on_route_to_original_destination'
PROPOSAL_COLLECTION_NAME = 'driver_proposals'

# Ride Request States
STATUS_PENDING = 'pending'
STATUS_PROPOSED = 'proposed_to_driver'
STATUS_ACCEPTED = 'accepted'
STATUS_ON_ROUTE_TO_PICKUP = 'on_route_to_pickup'

# Proposal Status States (for driver/rider app)
PROPOSAL_PENDING = 'pending_acceptance'
PROPOSAL_ACCEPTED = 'accepted'
PROPOSAL_REJECTED = 'rejected'

# --- Utility Functions ---

def initialize_firebase_app(cred_path, name):
    """Initializes a single Firebase app instance."""
    if not os.path.exists(cred_path):
        print(f"Error: Firebase credential file not found at {cred_path}", file=sys.stderr)
        return None
    try:
        cred = credentials.Certificate(cred_path)
        app = None
        try:
            app = firebase_admin.get_app(name)
        except ValueError:
            app = initialize_app(cred, name=name)
        
        db_client = firestore.client(app)
        print(f"Matcher Server: {name.capitalize()} DB initialized successfully.")
        return db_client
    except Exception as e:
        print(f"Error initializing Firebase App '{name}': {e}", file=sys.stderr)
        return None

def calculate_distance(lat1, lon1, lat2, lon2):
    """Calculate the distance between two coordinates in kilometers."""
    R = 6371
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)
    
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    return R * c

def calculate_dynamic_pickup(driver_loc, passenger_loc):
    """Calculates a dynamic pickup point if within threshold."""
    dist = calculate_distance(
        driver_loc.latitude, driver_loc.longitude,
        passenger_loc.latitude, passenger_loc.longitude
    )
    
    if dist < DYNAMIC_PICKUP_THRESHOLD_KM:
        mid_lat = (driver_loc.latitude + passenger_loc.latitude) / 2
        mid_lon = (driver_loc.longitude + passenger_loc.longitude) / 2
        print(f"  [PICKUP LOGIC] Dynamic midpoint calculated ({dist:.2f} km apart).")
        return GeoPoint(mid_lat, mid_lon)
    else:
        print(f"  [PICKUP LOGIC] Passenger location used ({dist:.2f} km apart).")
        return passenger_loc

def check_destination_overlap(driver_dest, passenger_dest):
    """Checks if passenger destination is near driver destination."""
    dist = calculate_distance(
        driver_dest.latitude, driver_dest.longitude,
        passenger_dest.latitude, passenger_dest.longitude
    )
    
    if dist <= MAX_DESTINATION_DEVIATION_KM:
        print(f"  [OVERLAP CHECK] Destination overlap confirmed ({dist:.2f} km deviation).")
        return True
    else:
        print(f"  [OVERLAP CHECK] Destination too far apart ({dist:.2f} km deviation).")
        return False

# --- Negotiation & Finalization Handlers ---

def finalize_match(pdb, ddb, proposal_data):
    """Executed when driver ACCEPTS the proposed ride."""
    ride_id = proposal_data['rideId']
    driver_id = proposal_data['driverId']
    final_pickup_loc = proposal_data['pickupLocation']
    rider_uid = proposal_data['riderUid']
    passenger_dest_loc = proposal_data['destinationLocation']
    vehicle_type = proposal_data['vehicleType']

    print(f"\n[ACCEPTANCE] Finalizing match for Ride {ride_id} with Driver {driver_id}.")

    try:
        # 1. Update Passenger DB ('ride_requests') to ACCEPTED
        ride_ref = pdb.collection('ride_requests').document(ride_id)
        ride_ref.update({
            'status': STATUS_ACCEPTED, 
            'driverId': driver_id,
            'matched_at': firestore.SERVER_TIMESTAMP,
            'finalPickupLocation': final_pickup_loc,
            'driverName': proposal_data.get('driverName', 'Driver')
        })
        print(f"  [DB UPDATE] Passenger ride {ride_id} updated to '{STATUS_ACCEPTED}'.")

        # 2. Update Driver DB ('riders') - Driver is now heading to pick up new passenger
        driver_ref = ddb.collection(RIDER_COLLECTION_NAME).document(driver_id)
        driver_ref.update({
            'status': STATUS_ON_ROUTE_TO_PICKUP,
            'currentPassengerId': rider_uid,
            'pooledRideId': ride_id,
            'nextTargetLocation': final_pickup_loc
        })
        print(f"  [DB UPDATE] Driver {driver_id} status changed to '{STATUS_ON_ROUTE_TO_PICKUP}'.")
        
        # 3. Create Public Ride Request for driver app
        public_ref = pdb.collection('public_ride_requests').document(ride_id)
        public_ref.set({
            'rideId': ride_id,
            'driverId': driver_id,
            'riderUid': rider_uid,
            'pickupLocation': final_pickup_loc,
            'destinationLocation': passenger_dest_loc,
            'vehicleType': vehicle_type,
            'driverName': proposal_data.get('driverName', 'Driver'),
            'timestamp': firestore.SERVER_TIMESTAMP
        })
        print(f"  [DB UPDATE] Public ride request created for driver visibility.")

        # 4. Cleanup: Delete the proposal document
        ddb.collection(PROPOSAL_COLLECTION_NAME).document(ride_id).delete()
        print(f"  [DB UPDATE] Proposal {ride_id} deleted.")
        
    except Exception as e:
        print(f"[FATAL ERROR] Finalization failed for ride {ride_id}: {e}", file=sys.stderr)

def handle_rejection(pdb, ddb, proposal_data):
    """Executed when driver REJECTS the proposed ride."""
    ride_id = proposal_data['rideId']
    print(f"\n[REJECTION] Driver rejected Ride {ride_id}. Reverting status.")
    
    try:
        # 1. Update Passenger DB ('ride_requests') back to PENDING
        ride_ref = pdb.collection('ride_requests').document(ride_id)
        ride_ref.update({'status': STATUS_PENDING})
        print(f"  [DB UPDATE] Passenger ride {ride_id} reverted to '{STATUS_PENDING}'.")

        # 2. Cleanup: Delete the proposal document
        ddb.collection(PROPOSAL_COLLECTION_NAME).document(ride_id).delete()
        print(f"  [DB UPDATE] Proposal {ride_id} deleted.")
        
    except Exception as e:
        print(f"[FATAL ERROR] Rejection handling failed for ride {ride_id}: {e}", file=sys.stderr)

# --- Main Matching Logic ---

def match_driver(pdb, ddb, ride_request_snapshot):
    """Finds a suitable driver and initiates the negotiation phase."""
    ride_id = ride_request_snapshot.id
    request_data = ride_request_snapshot.to_dict()
    
    passenger_pickup_loc = request_data.get('pickupLocation')
    passenger_dest_loc = request_data.get('destinationLocation')
    preferred_vehicle = request_data.get('vehiclePreference', 'Any')
    rider_uid = request_data.get('passengerId')

    if not (passenger_pickup_loc and passenger_dest_loc):
        print(f"[ERROR] Ride {ride_id} is missing location data. Skipping.")
        return

    print("\n" + "="*50)
    print(f"[MATCHING] Searching for driver for Ride ID: {ride_id}")
    
    # Query for drivers eligible for pooling
    available_drivers_query = ddb.collection(RIDER_COLLECTION_NAME).where('status', '==', RIDER_MATCH_STATUS)
    potential_drivers = available_drivers_query.stream()
    driver_list = list(potential_drivers)
    
    total_drivers_checked = 0
    best_match = None
    min_pickup_dist = float('inf')

    for driver_doc in driver_list:
        driver_data = driver_doc.to_dict()
        driver_id = driver_doc.id
        
        driver_current_loc = driver_data.get('currentRouteStart')
        driver_dest_loc = driver_data.get('currentRouteEnd')
        driver_name = driver_data.get('name', 'Driver')

        if not (driver_current_loc and driver_dest_loc):
            continue

        total_drivers_checked += 1
        
        # Check Destination Overlap
        if not check_destination_overlap(driver_dest_loc, passenger_dest_loc):
            continue

        # Check Initial Proximity
        pickup_dist = calculate_distance(
            driver_current_loc.latitude, driver_current_loc.longitude,
            passenger_pickup_loc.latitude, passenger_pickup_loc.longitude
        )
        
        if pickup_dist > MAX_MATCH_DISTANCE_KM:
            print(f"  Skipping {driver_id}: Pickup too far ({pickup_dist:.2f} km).")
            continue

        # Check Vehicle Type Match
        driver_vehicle_type = driver_data.get('vehicleType', '').lower()
        if preferred_vehicle != 'Any' and preferred_vehicle.lower() not in driver_vehicle_type.lower():
            print(f"  Skipping {driver_id}: Vehicle preference mismatch.")
            continue

        # Greedy Selection: Choose closest driver
        if pickup_dist < min_pickup_dist:
            min_pickup_dist = pickup_dist
            best_match = {
                'driver_id': driver_id,
                'driver_data': driver_data,
                'driver_loc': driver_current_loc,
                'driver_dest': driver_dest_loc,
                'driver_name': driver_name
            }
            print(f"  Found best potential match {driver_id} ({driver_name}) at {pickup_dist:.2f} km.")

    print(f"  Total {total_drivers_checked} active drivers checked for pooling.")

    if best_match:
        # --- PROPOSAL PHASE ---
        driver_id = best_match['driver_id']
        driver_data = best_match['driver_data']
        
        # Calculate Dynamic Pickup Point
        final_pickup_loc = calculate_dynamic_pickup(
            best_match['driver_loc'], passenger_pickup_loc
        )

        print(f"\n[PROPOSAL] Proposing Ride {ride_id} to Driver {driver_id}...")
        
        try:
            # 1. Update Passenger DB ('ride_requests') status to proposed
            ride_ref = pdb.collection('ride_requests').document(ride_id)
            ride_ref.update({
                'status': STATUS_PROPOSED,
                'matchedDriverId': driver_id,
                'proposedAt': firestore.SERVER_TIMESTAMP
            })
            print(f"  [DB UPDATE] Passenger ride {ride_id} updated to '{STATUS_PROPOSED}'.")

            # 2. Create PROPOSAL document in Driver DB (Driver App listens here)
            proposal_ref = ddb.collection(PROPOSAL_COLLECTION_NAME).document(ride_id)
            
            # IMPORTANT: Include ALL necessary fields for the driver app
            proposal_ref.set({
                'rideId': ride_id,
                'driverId': driver_id,
                'driverName': best_match['driver_name'],
                'riderUid': rider_uid,
                'status': PROPOSAL_PENDING,  # Initial status
                'pickupLocation': final_pickup_loc,
                'destinationLocation': passenger_dest_loc,
                'vehicleType': driver_data.get('vehicleType', 'Any'),
                'passengerName': request_data.get('passengerName', 'Passenger'),
                'originalFare': request_data.get('fare', 0),
                'pooledFare': request_data.get('fare', 0) * 0.7,  # Example discount for pooling
                'createdAt': firestore.SERVER_TIMESTAMP,
                'proposalExpiresAt': firestore.SERVER_TIMESTAMP  # Add 2-minute expiration
            })
            print(f"  [DB UPDATE] Proposal sent to Driver {driver_id} via '{PROPOSAL_COLLECTION_NAME}'.")
            
        except Exception as e:
            print(f"[FATAL ERROR] Proposal failed for ride {ride_id}: {e}", file=sys.stderr)
    else:
        print(f"\n[NO MATCH] No suitable driver found for Ride {ride_id}.")

# --- Snapshot Listeners ---

def on_pending_request_snapshot(col_snapshot, changes, read_time):
    """Listener for new pending ride requests."""
    for change in changes:
        if change.type.name in ('ADDED', 'MODIFIED'):
            data = change.document.to_dict()
            if data and data.get('status') == STATUS_PENDING:
                print(f"[EVENT] New or reverted PENDING request detected: {change.document.id}")
                match_driver(db_passenger, db_driver, change.document)

def on_proposal_update_snapshot(col_snapshot, changes, read_time):
    """Listener for driver responses to proposals."""
    for change in changes:
        if change.type.name == 'MODIFIED':
            data = change.document.to_dict()
            previous_data = getattr(change.document, '_previous', {})
            
            # Only process if status actually changed to accepted/rejected
            current_status = data.get('status')
            previous_status = previous_data.get('status') if previous_data else None
            
            if (current_status != previous_status and 
                current_status in [PROPOSAL_ACCEPTED, PROPOSAL_REJECTED]):
                
                if current_status == PROPOSAL_ACCEPTED:
                    print(f"[EVENT] Driver accepted proposal: {change.document.id}")
                    finalize_match(db_passenger, db_driver, data)
                elif current_status == PROPOSAL_REJECTED:
                    print(f"[EVENT] Driver rejected proposal: {change.document.id}")
                    handle_rejection(db_passenger, db_driver, data)

# --- Main Execution ---

if __name__ == '__main__':
    db_passenger = initialize_firebase_app(PASSENGER_DB_CREDENTIALS, 'passenger_app')
    db_driver = initialize_firebase_app(DRIVER_DB_CREDENTIALS, 'driver_app')

    if not (db_passenger and db_driver):
        print("\nFATAL: One or both database connections failed. Exiting.", file=sys.stderr)
        sys.exit(1)

    print(f"Listening for new requests (Max Match Distance: {MAX_MATCH_DISTANCE_KM} km, Max Destination Deviation: {MAX_DESTINATION_DEVIATION_KM} km)...")
    
    try:
        # Listener 1: Watch for new 'pending' ride requests
        query_pending = db_passenger.collection('ride_requests').where('status', '==', STATUS_PENDING)
        query_pending.on_snapshot(on_pending_request_snapshot)
        
        # Listener 2: Watch for driver responses
        query_proposals = db_driver.collection(PROPOSAL_COLLECTION_NAME)
        query_proposals.on_snapshot(on_proposal_update_snapshot)
        
        print("-" * 55)
        print(f"Matcher Server: Two listeners started (Pending Requests & Driver Responses).")
        print("Press Ctrl+C to stop the server.")
        print("-" * 55)

        while True:
            time.sleep(1)

    except KeyboardInterrupt:
        print("\nMatcher Server stopped by user.")
        sys.exit(0)
    except Exception as e:
        print(f"[FATAL ERROR] Main loop failed: {e}", file=sys.stderr)
        sys.exit(1)

Matcher Server: Passenger_app DB initialized successfully.
Matcher Server: Driver_app DB initialized successfully.
Listening for new requests (Max Match Distance: 5.0 km, Max Destination Deviation: 5.0 km)...
-------------------------------------------------------
Matcher Server: Two listeners started (Pending Requests & Driver Responses).
Press Ctrl+C to stop the server.
-------------------------------------------------------


C:\Users\DELL\anaconda3\Lib\site-packages\google\cloud\firestore_v1\base_collection.py:304: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
